## Scraping  ["Absa Bank Kenya Plc", "Bamburi Cement Ltd", "Britam Holdings Plc", "British American Tobacco Kenya Plc", "Centum Investment Company", "Cooperative Bank of Kenya"] from Kenyan wallstreet first page

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_kenyan_wall_street(company_name):
    # Base URL for Kenyan Wall Street search
    base_url = "https://kenyanwallstreet.com/?s="
    search_url = base_url + company_name.replace(" ", "+")
    
    # Send a GET request to the search URL
    response = requests.get(search_url)
    if response.status_code != 200:
        print(f"Failed to fetch data for {company_name}. HTTP Status: {response.status_code}")
        return []

    # Parse the HTML content
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Find all articles based on the structure
    articles = soup.find_all("h3", class_="jeg_post_title")
    data = []

    for article in articles:
        try:
            # Extract the headline
            title = article.find("a").get_text(strip=True)
            # Extract the URL
            link = article.find("a")["href"]
            # Extract the publication date (next sibling or parent container)
            date_tag = article.find_next("div", class_="jeg_meta_date")
            date = date_tag.get_text(strip=True) if date_tag else "Date not found"
            
            # Append to the data list
            data.append({"Company": company_name, "Title": title, "URL": link, "Date": date})
        except AttributeError:
            # Skip if any key element is missing
            continue

    return data

# List of companies to scrape
companies = [
    "Absa Bank Kenya Plc", "Bamburi Cement Ltd", "Britam Holdings Plc",
    "British American Tobacco Kenya Plc", "Centum Investment Company", "Cooperative Bank of Kenya"
]

# Collect data for all companies
all_data = []

for company in companies:
    print(f"Scraping data for {company}...")
    results = scrape_kenyan_wall_street(company)
    all_data.extend(results)

# Save the data to a CSV file
df = pd.DataFrame(all_data)
df.to_csv("kenyan_wall_street_news.csv", index=False)

print("Scraping complete. Data saved to 'kenyan_wall_street_news.csv'.")


In [ ]:
kenyanwallstreetdf = pd.read_csv('kenyan_wall_street_news.csv')
kenyanwallstreetdf

## Scraping maximum of 5 pages 

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_kenyan_wall_street(company_name, max_pages=5):
    """
    Scrapes headlines, URLs, and publication dates for a given company from Kenyan Wall Street with pagination.

    Parameters:
        company_name (str): Name of the company to search for.
        max_pages (int): Maximum number of pages to scrape.

    Returns:
        list: A list of dictionaries containing scraped data.
    """
    base_url = "https://kenyanwallstreet.com"
    search_url = f"{base_url}/?s={company_name.replace(' ', '+')}"
    all_data = []

    for page in range(1, max_pages + 1):
        # Construct the paginated URL
        if page == 1:
            url = search_url  # Page 1 uses the base search URL
        else:
            url = f"{base_url}/page/{page}/?s={company_name.replace(' ', '+')}"

        print(f"Scraping Page {page}: {url}")
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch data from {url}. HTTP Status: {response.status_code}")
            break

        # Parse the HTML content
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Find all articles based on the structure
        articles = soup.find_all("h3", class_="jeg_post_title")
        
        for article in articles:
            try:
                # Extract headline
                title = article.find("a").get_text(strip=True)
                # Extract URL
                link = article.find("a")["href"]
                # Extract publication date
                date_tag = article.find_next("div", class_="jeg_meta_date")
                date = date_tag.get_text(strip=True) if date_tag else "Date not found"
                
                # Append to the data list
                all_data.append({
                    "Company": company_name,
                    "Title": title,
                    "URL": link,
                    "Date": date
                })
            except AttributeError:
                # Skip if any key element is missing
                continue

    return all_data

# List of companies to scrape
companies = [
    "Absa Bank Kenya Plc", "Bamburi Cement Ltd", "Britam Holdings Plc",
    "British American Tobacco Kenya Plc", "Centum Investment Company", "Cooperative Bank of Kenya"
]

# Scrape data for all companies
all_results = []

for company in companies:
    print(f"Starting scrape for: {company}")
    results = scrape_kenyan_wall_street(company, max_pages=5)  # Limit to 5 pages for PoC
    all_results.extend(results)

# Save the data to a CSV file
df = pd.DataFrame(all_results)
df.to_csv("kenyan_wall_street_paginated_news.csv", index=False)

print("Scraping complete. Data saved to 'kenyan_wall_street_paginated_news.csv'.")


In [ ]:
kenyanwallstreetpgdf = pd.read_csv('kenyan_wall_street_paginated_news.csv')
kenyanwallstreetpgdf.head()

In [ ]:
kenyanwallstreetpgdf.shape

In [ ]:
kenyanwallstreetpgdf

## Iterating through all pages

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_kenyan_wall_street(company_name):
    """
    Scrapes headlines, URLs, and publication dates for a given company from Kenyan Wall Street with dynamic pagination.

    Parameters:
        company_name (str): Name of the company to search for.

    Returns:
        list: A list of dictionaries containing scraped data.
    """
    base_url = "https://kenyanwallstreet.com"
    search_url = f"{base_url}/?s={company_name.replace(' ', '+')}"
    all_data = []

    # Get the total number of pages dynamically
    print(f"Fetching total page count for {company_name}...")
    response = requests.get(search_url)
    if response.status_code != 200:
        print(f"Failed to fetch the search page for {company_name}. HTTP Status: {response.status_code}")
        return []
    
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Find the total number of pages
    total_pages_tag = soup.find("span", class_="page_info")
    try:
        total_pages = int(total_pages_tag.get_text(strip=True).split()[-1]) if total_pages_tag else 1
    except (ValueError, AttributeError):
        total_pages = 1

    print(f"Total pages found: {total_pages}")
    
    # Iterate through all pages
    for page in range(1, total_pages + 1):
        if page == 1:
            url = search_url  # Page 1 uses the base search URL
        else:
            url = f"{base_url}/page/{page}/?s={company_name.replace(' ', '+')}"

        print(f"Scraping Page {page}: {url}")
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch data from {url}. HTTP Status: {response.status_code}")
            break

        # Parse the HTML content
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Find all articles based on the structure
        articles = soup.find_all("h3", class_="jeg_post_title")
        
        for article in articles:
            try:
                # Extract headline
                title = article.find("a").get_text(strip=True)
                # Extract URL
                link = article.find("a")["href"]
                # Extract publication date
                date_tag = article.find_next("div", class_="jeg_meta_date")
                date = date_tag.get_text(strip=True) if date_tag else "Date not found"
                
                # Append to the data list
                all_data.append({
                    "Company": company_name,
                    "Title": title,
                    "URL": link,
                    "Date": date
                })
            except AttributeError:
                # Skip if any key element is missing
                continue

    return all_data

# List of companies to scrape
companies = [
    "Absa Bank Kenya Plc", "Bamburi Cement Ltd", "Britam Holdings Plc",
    "British American Tobacco Kenya Plc", "Centum Investment Company", "Cooperative Bank of Kenya"
]

# Scrape data for all companies
all_results = []

for company in companies:
    print(f"Starting scrape for: {company}")
    results = scrape_kenyan_wall_street(company)
    all_results.extend(results)

# Save the data to a CSV file
df = pd.DataFrame(all_results)
df.to_csv("kenyan_wall_street_dynamic_paginated_news.csv", index=False)

print("Scraping complete. Data saved to 'kenyan_wall_street_dynamic_paginated_news.csv'.")


In [ ]:
allpagesdf = pd.read_csv('kenyan_wall_street_dynamic_paginated_news.csv')
allpagesdf

## Articles with content 

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_article_content(article_url):
    """
    Fetches the content of an article from its URL.
    
    Parameters:
        article_url (str): The URL of the article to fetch.
    
    Returns:
        str: The extracted content of the article, or an empty string if content is unavailable.
    """
    try:
        response = requests.get(article_url)
        if response.status_code != 200:
            print(f"Failed to fetch article content from {article_url}. HTTP Status: {response.status_code}")
            return ""
        
        # Parse the article page
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Extract the article content
        content_div = soup.find("div", class_="content-inner")
        if content_div:
            paragraphs = content_div.find_all("p")  # Extract all <p> tags within the content-inner div
            content = " ".join([para.get_text(strip=True) for para in paragraphs])
            return content
        else:
            print(f"Article content not found for URL: {article_url}")
            return ""
    except Exception as e:
        print(f"Error fetching article content: {e}")
        return ""

def scrape_kenyan_wall_street(company_name):
    """
    Scrapes headlines, URLs, publication dates, and article content for a given company from Kenyan Wall Street.

    Parameters:
        company_name (str): Name of the company to search for.

    Returns:
        list: A list of dictionaries containing scraped data.
    """
    base_url = "https://kenyanwallstreet.com"
    search_url = f"{base_url}/?s={company_name.replace(' ', '+')}"
    all_data = []

    # Get the total number of pages dynamically
    print(f"Fetching total page count for {company_name}...")
    response = requests.get(search_url)
    if response.status_code != 200:
        print(f"Failed to fetch the search page for {company_name}. HTTP Status: {response.status_code}")
        return []
    
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Find the total number of pages
    total_pages_tag = soup.find("span", class_="page_info")
    try:
        total_pages = int(total_pages_tag.get_text(strip=True).split()[-1]) if total_pages_tag else 1
    except (ValueError, AttributeError):
        total_pages = 1

    print(f"Total pages found: {total_pages}")
    
    # Iterate through all pages
    for page in range(1, total_pages + 1):
        if page == 1:
            url = search_url  # Page 1 uses the base search URL
        else:
            url = f"{base_url}/page/{page}/?s={company_name.replace(' ', '+')}"

        print(f"Scraping Page {page}: {url}")
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch data from {url}. HTTP Status: {response.status_code}")
            break

        # Parse the HTML content
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Find all articles based on the structure
        articles = soup.find_all("h3", class_="jeg_post_title")
        
        for article in articles:
            try:
                # Extract headline
                title = article.find("a").get_text(strip=True)
                # Extract URL
                link = article.find("a")["href"]
                # Extract publication date
                date_tag = article.find_next("div", class_="jeg_meta_date")
                date = date_tag.get_text(strip=True) if date_tag else "Date not found"
                # Fetch the article content
                content = get_article_content(link)
                
                # Append to the data list
                all_data.append({
                    "Company": company_name,
                    "Title": title,
                    "URL": link,
                    "Date": date,
                    "Content": content
                })
            except AttributeError:
                # Skip if any key element is missing
                continue

    return all_data

# List of companies to scrape
companies = [
    "Absa Bank Kenya Plc", "Bamburi Cement Ltd", "Britam Holdings Plc",
    "British American Tobacco Kenya Plc", "Centum Investment Company", "Cooperative Bank of Kenya","BK Group Plc",
    "BOC Kenya Plc","Car and General (K) Ltd","Carbacid Investments Plc","CIC Insurance Group Ltd","Crown Paints Kenya Plc",
    "Diamond Trust Bank Kenya Ltd","Eaagads Ltd","East African Breweries Ltd","East African Cables Ltd","East African Portland Cement Ltd",
    "Equity Group Holdings Plc","Eveready East Africa Ltd","Homeboyz Entertainment Plc","I & M Holdings Plc","Jubilee Holdings Ltd",
    "Kakuzi Plc","KCB Group Plc","Kenya Power and Lighting Company Plc","Nation Media Group Plc","NCBA Group Plc","Safaricom Plc",
    "Stanbic Holdings Plc","Standard Chartered Bank Kenya Ltd","Standard Group Plc","Uchumi Supermarket Plc","Unilever Tea Kenya Ltd",
    "Unga Group Ltd"
]

# Scrape data for all companies
all_results = []

for company in companies:
    print(f"Starting scrape for: {company}")
    results = scrape_kenyan_wall_street(company)
    all_results.extend(results)

# Save the data to a CSV file
df = pd.DataFrame(all_results)
df.to_csv("New_kenyan_wall_street_with_content.csv", index=False)

print("Scraping complete. Data saved to 'New_kenyan_wall_street_with_content.csv'.")


In [ ]:
content_df = pd.read_csv('New_kenyan_wall_street_with_content.csv')
content_df

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import quote  # Import for URL encoding

def get_article_content(article_url):
    """
    Fetches the content of an article from its URL.
    
    Parameters:
        article_url (str): The URL of the article to fetch.
    
    Returns:
        str: The extracted content of the article, or an empty string if content is unavailable.
    """
    try:
        response = requests.get(article_url)
        if response.status_code != 200:
            print(f"Failed to fetch article content from {article_url}. HTTP Status: {response.status_code}")
            return ""
        
        # Parse the article page
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Extract the article content
        content_div = soup.find("div", class_="content-inner")
        if content_div:
            paragraphs = content_div.find_all("p")  # Extract all <p> tags within the content-inner div
            content = " ".join([para.get_text(strip=True) for para in paragraphs])
            return content
        else:
            print(f"Article content not found for URL: {article_url}")
            return ""
    except Exception as e:
        print(f"Error fetching article content: {e}")
        return ""

def scrape_kenyan_wall_street(company_name):
    """
    Scrapes headlines, URLs, publication dates, and article content for a given company from Kenyan Wall Street.

    Parameters:
        company_name (str): Name of the company to search for.

    Returns:
        list: A list of dictionaries containing scraped data.
    """
    base_url = "https://kenyanwallstreet.com"
    encoded_company_name = quote(company_name)  # Encode the company name for the URL
    search_url = f"{base_url}/?s={encoded_company_name}"
    all_data = []

    # Get the total number of pages dynamically
    print(f"Fetching total page count for {company_name}...")
    response = requests.get(search_url)
    if response.status_code != 200:
        print(f"Failed to fetch the search page for {company_name}. HTTP Status: {response.status_code}")
        return []
    
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Find the total number of pages
    total_pages_tag = soup.find("span", class_="page_info")
    try:
        total_pages = int(total_pages_tag.get_text(strip=True).split()[-1]) if total_pages_tag else 1
    except (ValueError, AttributeError):
        total_pages = 1

    print(f"Total pages found: {total_pages}")
    
    # Iterate through all pages
    for page in range(1, total_pages + 1):
        if page == 1:
            url = search_url  # Page 1 uses the base search URL
        else:
            url = f"{base_url}/page/{page}/?s={encoded_company_name}"

        print(f"Scraping Page {page}: {url}")
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch data from {url}. HTTP Status: {response.status_code}")
            break

        # Parse the HTML content
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Find all articles based on the structure
        articles = soup.find_all("h3", class_="jeg_post_title")
        
        for article in articles:
            try:
                # Extract headline
                title = article.find("a").get_text(strip=True)
                # Extract URL
                link = article.find("a")["href"]
                # Extract publication date
                date_tag = article.find_next("div", class_="jeg_meta_date")
                date = date_tag.get_text(strip=True) if date_tag else "Date not found"
                # Fetch the article content
                content = get_article_content(link)
                
                # Append to the data list
                all_data.append({
                    "Company": company_name,
                    "Title": title,
                    "URL": link,
                    "Date": date,
                    "Content": content
                })
            except AttributeError:
                # Skip if any key element is missing
                continue

    return all_data

# List of companies to scrape
companies = [
    "Absa Bank Kenya Plc", "Bamburi Cement Ltd", "Britam Holdings Plc",
    "British American Tobacco Kenya Plc", "Centum Investment Company", "Cooperative Bank of Kenya","BK Group Plc",
    "BOC Kenya Plc","Car and General (K) Ltd","Carbacid Investments Plc","CIC Insurance Group Ltd","Crown Paints Kenya Plc",
    "Diamond Trust Bank Kenya Ltd","Eaagads Ltd","East African Breweries Ltd","East African Cables Ltd","East African Portland Cement Ltd",
    "Equity Group Holdings Plc","Eveready East Africa Ltd","Homeboyz Entertainment Plc","I & M Holdings Plc","Jubilee Holdings Ltd",
    "Kakuzi Plc","KCB Group Plc","Kenya Power and Lighting Company Plc","Nation Media Group Plc","NCBA Group Plc","Safaricom Plc",
    "Stanbic Holdings Plc","Standard Chartered Bank Kenya Ltd","Standard Group Plc","Uchumi Supermarket Plc","Unilever Tea Kenya Ltd",
    "Unga Group Ltd"
]

# Scrape data for all companies
all_results = []

for company in companies:
    print(f"Starting scrape for: {company}")
    results = scrape_kenyan_wall_street(company)
    all_results.extend(results)

# Save the data to a CSV file
df = pd.DataFrame(all_results)
df.to_csv("kenyan_wall_street_with_content.csv", index=False)

print("Scraping complete. Data saved to 'kenyan_wall_street_with_content.csv'.")


Starting scrape for: Absa Bank Kenya Plc
Fetching total page count for Absa Bank Kenya Plc...
Total pages found: 14
Scraping Page 1: https://kenyanwallstreet.com/?s=Absa%20Bank%20Kenya%20Plc
Scraping Page 2: https://kenyanwallstreet.com/page/2/?s=Absa%20Bank%20Kenya%20Plc
Scraping Page 3: https://kenyanwallstreet.com/page/3/?s=Absa%20Bank%20Kenya%20Plc
Scraping Page 4: https://kenyanwallstreet.com/page/4/?s=Absa%20Bank%20Kenya%20Plc
Scraping Page 5: https://kenyanwallstreet.com/page/5/?s=Absa%20Bank%20Kenya%20Plc
Scraping Page 6: https://kenyanwallstreet.com/page/6/?s=Absa%20Bank%20Kenya%20Plc
Scraping Page 7: https://kenyanwallstreet.com/page/7/?s=Absa%20Bank%20Kenya%20Plc
Scraping Page 8: https://kenyanwallstreet.com/page/8/?s=Absa%20Bank%20Kenya%20Plc
Scraping Page 9: https://kenyanwallstreet.com/page/9/?s=Absa%20Bank%20Kenya%20Plc
Scraping Page 10: https://kenyanwallstreet.com/page/10/?s=Absa%20Bank%20Kenya%20Plc
Scraping Page 11: https://kenyanwallstreet.com/page/11/?s=Absa%20Bank

Scraping Page 2: https://kenyanwallstreet.com/page/2/?s=Crown%20Paints%20Kenya%20Plc
Scraping Page 3: https://kenyanwallstreet.com/page/3/?s=Crown%20Paints%20Kenya%20Plc
Scraping Page 4: https://kenyanwallstreet.com/page/4/?s=Crown%20Paints%20Kenya%20Plc
Scraping Page 5: https://kenyanwallstreet.com/page/5/?s=Crown%20Paints%20Kenya%20Plc
Starting scrape for: Diamond Trust Bank Kenya Ltd
Fetching total page count for Diamond Trust Bank Kenya Ltd...
Total pages found: 2
Scraping Page 1: https://kenyanwallstreet.com/?s=Diamond%20Trust%20Bank%20Kenya%20Ltd
Scraping Page 2: https://kenyanwallstreet.com/page/2/?s=Diamond%20Trust%20Bank%20Kenya%20Ltd
Starting scrape for: Eaagads Ltd
Fetching total page count for Eaagads Ltd...
Total pages found: 1
Scraping Page 1: https://kenyanwallstreet.com/?s=Eaagads%20Ltd
Starting scrape for: East African Breweries Ltd
Fetching total page count for East African Breweries Ltd...
Total pages found: 3
Scraping Page 1: https://kenyanwallstreet.com/?s=East%20A

Total pages found: 6
Scraping Page 1: https://kenyanwallstreet.com/?s=Kenya%20Power%20and%20Lighting%20Company%20Plc
Scraping Page 2: https://kenyanwallstreet.com/page/2/?s=Kenya%20Power%20and%20Lighting%20Company%20Plc
Scraping Page 3: https://kenyanwallstreet.com/page/3/?s=Kenya%20Power%20and%20Lighting%20Company%20Plc
Scraping Page 4: https://kenyanwallstreet.com/page/4/?s=Kenya%20Power%20and%20Lighting%20Company%20Plc
Scraping Page 5: https://kenyanwallstreet.com/page/5/?s=Kenya%20Power%20and%20Lighting%20Company%20Plc
Scraping Page 6: https://kenyanwallstreet.com/page/6/?s=Kenya%20Power%20and%20Lighting%20Company%20Plc
Starting scrape for: Nation Media Group Plc
Fetching total page count for Nation Media Group Plc...
Total pages found: 15
Scraping Page 1: https://kenyanwallstreet.com/?s=Nation%20Media%20Group%20Plc
Scraping Page 2: https://kenyanwallstreet.com/page/2/?s=Nation%20Media%20Group%20Plc
Scraping Page 3: https://kenyanwallstreet.com/page/3/?s=Nation%20Media%20Group%20Pl

Scraping Page 16: https://kenyanwallstreet.com/page/16/?s=Standard%20Group%20Plc
Starting scrape for: Uchumi Supermarket Plc
Fetching total page count for Uchumi Supermarket Plc...
Total pages found: 6
Scraping Page 1: https://kenyanwallstreet.com/?s=Uchumi%20Supermarket%20Plc
Scraping Page 2: https://kenyanwallstreet.com/page/2/?s=Uchumi%20Supermarket%20Plc
Scraping Page 3: https://kenyanwallstreet.com/page/3/?s=Uchumi%20Supermarket%20Plc
Scraping Page 4: https://kenyanwallstreet.com/page/4/?s=Uchumi%20Supermarket%20Plc
Scraping Page 5: https://kenyanwallstreet.com/page/5/?s=Uchumi%20Supermarket%20Plc
Scraping Page 6: https://kenyanwallstreet.com/page/6/?s=Uchumi%20Supermarket%20Plc
Starting scrape for: Unilever Tea Kenya Ltd
Fetching total page count for Unilever Tea Kenya Ltd...
Total pages found: 1
Scraping Page 1: https://kenyanwallstreet.com/?s=Unilever%20Tea%20Kenya%20Ltd
Starting scrape for: Unga Group Ltd
Fetching total page count for Unga Group Ltd...
Total pages found: 1
Scr

In [3]:
data_df = pd.read_csv('kenyan_wall_street_with_content.csv')
data_df

,Company,Title,URL,Date,Content
0,Absa Bank Kenya Plc,Absa Bank Kenya Plc Profit up 14% On Strong Ba...,https://kenyanwallstreet.com/absa-bank-kenya-p...,"November 24, 2023",Absa Bank Kenya posted a net profit of KSh 12....
1,Absa Bank Kenya Plc,Absa Bank Kenya PLC commits Ksh100 billion to ...,https://kenyanwallstreet.com/absa-bank-kenya-p...,"October 18, 2023",Absa Bank Kenya has announced a Ksh100 billion...
2,Absa Bank Kenya Plc,Absa Bank Kenya PLC records 161% Rise in Profi...,https://kenyanwallstreet.com/absa-bank-kenya-r...,"March 15, 2022",Absa Bank Kenya PLChas today announced a growt...
3,Absa Bank Kenya Plc,Absa Bank Kenya Plc Media Statement,https://kenyanwallstreet.com/absa-bank-kenya-p...,"April 10, 2020",We have noted the Central Bank of Kenya’spress...
4,Absa Bank Kenya Plc,"HSBC to Exit South Africa in Deals with Absa, ...",https://kenyanwallstreet.com/hsbc-to-exit-sout...,"September 27, 2024",Multinational British lender HSBC Holdings has...
...,...,...,...,...,...
2490,Unga Group Ltd,April Monthly Report,https://kenyanwallstreet.com/april-monthly-rep...,"January 11, 2017",‘More money is made on Wall Street from salesm...
2491,Unga Group Ltd,Q1 2016 Market Analysis: ‘Back To Where We Began’,https://kenyanwallstreet.com/q1-2016-market-an...,"April 4, 2016","Dear Investor, As we usher in the second quart..."
2492,Unga Group Ltd,Kakuzi Limited Analysis,https://kenyanwallstreet.com/kakuzi-limited-an...,"January 11, 2017",Profile Kakuzi Limited is a Kenya-based compan...
2493,Unga Group Ltd,Centum Investment Company Analysis,https://kenyanwallstreet.com/centum-investment...,"May 18, 2017",History ICDC was founded in 1954 as a governme...
